In [ ]:
#default_exp utils

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# utils

> Various utility functions used by the blurr package.

In [ ]:
#export
import importlib, inspect, sys, torch

import numpy as np
import pandas as pd

from enum import Enum
from fastcore.foundation import L
from transformers import logging, AutoConfig, AutoTokenizer
from typing import List, Union, Type

logging.set_verbosity_error()

In [ ]:
#hide_input
import os, pdb

from nbverbose.showdoc import *
from fastcore.test import *

In [ ]:
#hide_input
#cuda
torch.cuda.set_device(1)
print(f'Using GPU #{torch.cuda.current_device()}: {torch.cuda.get_device_name()}')

Using GPU #1: GeForce GTX 1080 Ti


## Utility classes and methods

The `Singleton` class and `str_to_type` methods are used in the construction of the `BLURR` instance, as well as, in other parts of the library. `print_versions` is a nicety added for developers wishing to know what versions of specific libraries are being used for documentation and troubleshooting.

In [ ]:
#export
class Singleton:
    def __init__(self, cls):
        self._cls, self._instance = cls, None

    def __call__(self, *args, **kwargs):
        if self._instance == None: self._instance = self._cls(*args, **kwargs)
        return self._instance

`Singleton` functions as python decorator.  Use this above any class to turn that class into a singleton (see [here](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Singleton.html) for more info on the singleton pattern).

In [ ]:
@Singleton
class TestSingleton: pass

a = TestSingleton()
b = TestSingleton()
test_eq(a,b)

In [ ]:
#export
def str_to_type(
    typename:str  # The name of a type as a string
) -> Type:        # Returns the actual type
    "Converts a type represented as a string to the actual class"
    return getattr(sys.modules[__name__], typename)

In [ ]:
show_doc(str_to_type)

<h4 id="str_to_type" class="doc_header"><code>str_to_type</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>str_to_type</code>(**`typename`**:`str`)

Converts a type represented as a string to the actual class

**Parameters:**


 - **`typename`** : *`<class 'str'>`*	<p>The name of a type as a string</p>



**Returns**:
	
 * *`typing.Type`*	<p>Returns the actual type</p>



How to use:

In [ ]:
print(str_to_type('List'))
print(str_to_type('test_eq'))
print(str_to_type('TestSingleton'))

typing.List
<function test_eq at 0x7f2fce278af0>


In [ ]:
#export
def print_versions(
    # A string of space delimited package names or a list of package names
    packages:Union[str, List[str]] 
):   
    """ Prints the name and version of one or more packages in your environment"""
    packages = packages.split(' ') if isinstance(packages, str) else packages

    for item in packages:
        item = item.strip()
        print(f'{item}: {importlib.import_module(item).__version__}')

In [ ]:
show_doc(print_versions)

<h4 id="print_versions" class="doc_header"><code>print_versions</code><a href="__main__.py#L2" class="source_link" style="float:right">[source]</a></h4>

> <code>print_versions</code>(**`packages`**:`Union`\[`str`, `List`\[`str`\]\])

Prints the name and version of one or more packages in your environment

**Parameters:**


 - **`packages`** : *`typing.Union[str, typing.List[str]]`*	<p>A string of space delimited package names or a list of package names</p>



How to use:

In [ ]:
print_versions('torch transformers fastai')
print('---')
print_versions(['torch', 'transformers', 'fastai'])

torch: 1.7.1
transformers: 4.9.2
fastai: 2.5.0
---
torch: 1.7.1
transformers: 4.9.2
fastai: 2.5.0


## BlurrUtil

In [ ]:
#export
@Singleton
class BlurrUtil():
    """A general utility class for getting your Hugging Face objects"""
    def __init__(self):
        # get hf classes (tokenizers, configs, models, etc...)
        transformer_classes = inspect.getmembers(importlib.import_module('transformers'))
        
        # build a df that we can query against to get various transformers objects/info
        self._df = pd.DataFrame(transformer_classes, columns=['class_name', 'class_location'])
        self._df = self._df[self._df.class_location.apply(lambda v: isinstance(v, type))]
        
        # add the module each class is included in
        self._df['module'] = self._df.class_location.apply(lambda v: v.__module__)
        
        # remove class_location (don't need it anymore)
        self._df.drop(labels=['class_location'], axis=1, inplace=True)
        
        # break up the module into separate cols
        module_parts_df = self._df.module.str.split(".", n = -1, expand = True) 
        for i in range(len(module_parts_df.columns)):
            self._df[f'module_part_{i}'] = module_parts_df[i]

        # using module part 1, break up the functional area and arch into separate cols
        module_part_3_df = self._df.module_part_3.str.split("_", n = 1, expand = True) 
        self._df[['functional_area', 'arch']] = module_part_3_df
        
        # transformers >=4.5.x does "auto" differently; so remove it and "utils" from "arch" column
        self._df = self._df[~self._df['arch'].isin(['auto', 'utils'])]
        

        # if functional area = modeling, pull out the task it is built for
        model_type_df = self._df[(self._df.functional_area == 'modeling')].class_name.str.rsplit('For', 
                                                                                                 n=1, 
                                                                                                 expand=True)
        
        model_type_df[1] = np.where(model_type_df[1].notnull(), 
                                    'For' + model_type_df[1].astype(str), 
                                    model_type_df[1])
        
        self._df['model_task'] = model_type_df[1]
        self._df['model_task'] = self._df['model_task'].str.replace('For', '', n=1, case=True, regex=False)
        
        model_type_df = self._df[(self._df.functional_area == 'modeling')].class_name.str.rsplit('With', 
                                                                                                 n=1, 
                                                                                                 expand=True)
        model_type_df[1] = np.where(model_type_df[1].notnull(), 
                                    'With' + model_type_df[1].astype(str), 
                                    self._df[(self._df.functional_area == 'modeling')].model_task)
        
        self._df['model_task'] = model_type_df[1]
        self._df['model_task'] = self._df['model_task'].str.replace('With', '', n=1, case=True, regex=False)
        
        # look at what we're going to remove (use to verify we're just getting rid of stuff we want too)
        # df[~df['hf_class_type'].isin(['modeling', 'configuration', 'tokenization'])]
        
        # only need these 3 functional areas for our querying purposes
        self._df = self._df[self._df['functional_area'].isin(['modeling', 'configuration', 'tokenization'])]
       
    def get_tasks(
        self, 
        arch:str=None # A transformer architecture (e.g., 'bert')
    ):                # A list of tasks you can use
        """This method can be used to get a list of all tasks supported by your transformers install, or
        just those available to a specific architecture
        """
        query = ['model_task.notna()']
        if (arch): query.append(f'arch == "{arch}"')

        return sorted(self._df.query(' & '.join(query), engine='python').model_task.unique().tolist())
    
    def get_architectures(
        self
    ):            # Returns a list of architectures supported by your transformers install
        return sorted(self._df[(self._df.arch.notna()) & 
                        (self._df.arch != None)].arch.unique().tolist())
    
    def get_models(
        self, 
        arch:str=None, # A transformer architecture (e.g., 'bert')
        task:str=None  # A transformer task (e.g., 'TokenClassification')
    ):
        """The transformer models available for use (optional: by architecture | task)"""
        query = ['functional_area == "modeling"']
        if (arch): query.append(f'arch == "{arch}"')
        if (task): query.append(f'model_task == "{task}"')

        models = sorted(self._df.query(' & '.join(query)).class_name.tolist())
        return models
    
    def get_model_architecture(
        self, 
        model_name_or_enum
    ):
        """Get the architecture for a given model name / enum"""
        model_name = model_name_or_enum if isinstance(model_name_or_enum, str) else model_name_or_enum.name
        return self._df[self._df.class_name == model_name].arch.values[0]
    
    def get_hf_objects(
        self, 
        # The name or path of the pretrained model
        pretrained_model_name_or_path, 
        # The model class you want to use (e.g., AutoModelFor<task>)
        model_cls,       
        # A specific configuration instance you want to use. If None, a configuration object will be instantiated
        # using the AutoConfig class along with any supplied `config_kwargs`
        config=None,      
        # A specific tokenizer instance you want to use.
        tokenizer_cls=None,         
        # Any kwargs you want to pass to the `AutoConfig` (only used if you do NOT pass int a config above)
        config_kwargs={},  
        # Any kwargs you want to pass in the creation of your tokenizer
        tokenizer_kwargs={},    
        # Any kwargs you want to pass in the creation of your model
        model_kwargs={},               
         # If you want to change the location Hugging Face objects are cached
        cache_dir=None                
    ):   # A tuple (architecture (str), config (obj), tokenizer (obj), and model (obj) 
        """Given at minimum a `pretrained_model_name_or_path` and `model_cls (such as 
        `AutoModelForSequenceClassification"), this method returns all the Hugging Face objects you need to train
        a model using Blurr
        """
        # config
        if (config is None):
            config = AutoConfig.from_pretrained(pretrained_model_name_or_path, 
                                                cache_dir=cache_dir, 
                                                **config_kwargs)
        
        # tokenizer (gpt2, roberta, bart (and maybe others) tokenizers require a prefix space)
        if (any(s in pretrained_model_name_or_path for s in ['gpt2', 'roberta', 'bart', 'longformer'])):
            tokenizer_kwargs = { **{'add_prefix_space': True}, **tokenizer_kwargs }

        if (tokenizer_cls is None):
            tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path, 
                                                      cache_dir=cache_dir, 
                                                      **tokenizer_kwargs)
        else:
            tokenizer = tokenizer_cls.from_pretrained(pretrained_model_name_or_path, 
                                                      cache_dir=cache_dir, 
                                                      **tokenizer_kwargs)
            
        # model        
        model = model_cls.from_pretrained(pretrained_model_name_or_path, 
                                          config=config, 
                                          cache_dir=cache_dir, 
                                          **model_kwargs)

        #arch
        arch = self.get_model_architecture(type(model).__name__)
        
        return (arch, config, tokenizer, model)

`BlurrUtil` is a `Singleton` (there exists only one instance, and the same instance is returned upon subsequent instantiation requests).  You can get at via the `BLURR` constant below.

In [ ]:
mh = BlurrUtil()
mh2 = BlurrUtil()
test_eq(mh, mh2)

In [ ]:
#hide
display(mh._df.head())

,class_name,module,module_part_0,module_part_1,module_part_2,module_part_3,functional_area,arch,model_task
6,AdaptiveEmbedding,transformers.models.transfo_xl.modeling_transf...,transformers,models,transfo_xl,modeling_transfo_xl,modeling,transfo_xl,None
8,AlbertConfig,transformers.models.albert.configuration_albert,transformers,models,albert,configuration_albert,configuration,albert,NaN
9,AlbertForMaskedLM,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,MaskedLM
10,AlbertForMultipleChoice,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,MultipleChoice
11,AlbertForPreTraining,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,PreTraining


In [ ]:
#hide
display(mh._df.head())

print(list(mh._df.model_task.unique()))
print('')
print(list(mh._df.functional_area.unique()))
print('')
print(list(mh._df.arch.unique()))
print('')
print(list(mh._df.module_part_3.unique()))

,class_name,module,module_part_0,module_part_1,module_part_2,module_part_3,functional_area,arch,model_task
6,AdaptiveEmbedding,transformers.models.transfo_xl.modeling_transf...,transformers,models,transfo_xl,modeling_transfo_xl,modeling,transfo_xl,None
8,AlbertConfig,transformers.models.albert.configuration_albert,transformers,models,albert,configuration_albert,configuration,albert,NaN
9,AlbertForMaskedLM,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,MaskedLM
10,AlbertForMultipleChoice,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,MultipleChoice
11,AlbertForPreTraining,transformers.models.albert.modeling_albert,transformers,models,albert,modeling_albert,modeling,albert,PreTraining


[None, nan, 'MaskedLM', 'MultipleChoice', 'PreTraining', 'QuestionAnswering', 'SequenceClassification', 'TokenClassification', 'CausalLM', 'ConditionalGeneration', 'NextSentencePrediction', 'ImageClassification', 'Teacher', 'QuestionAnsweringSimple', 'LMHeadModel', 'CTC', 'EntityClassification', 'EntityPairClassification', 'EntitySpanClassification', 'Classification', 'Generation', 'LMHead', 'merLayer', 'merModel', 'merPreTrainedModel', 'RegionToPhraseAlignment', 'VisualReasoning']

['modeling', 'configuration', 'tokenization']

['transfo_xl', 'albert', 'albert_fast', 'bart', 'bart_fast', 'barthez', 'barthez_fast', 'bert', 'bert_generation', 'bert_japanese', 'bert_fast', 'bertweet', 'big_bird', 'bigbird_pegasus', 'big_bird_fast', 'blenderbot', 'blenderbot_small', 'byt5', 'clip', 'clip_fast', 'ctrl', 'camembert', 'camembert_fast', 'canine', 'convbert', 'convbert_fast', 'cpm', 'dpr', 'dpr_fast', 'deberta', 'deberta_fast', 'deberta_v2', 'deit', 'detr', 'distilbert', 'distilbert_fast', 'el

### Provide global helper constant

Users of this library can simply use `BLURR` to access all the `BlurrUtil` capabilities without having to fetch an instance themselves.

In [ ]:
#export
BLURR = BlurrUtil()

Here's how you can get at the core Hugging Face objects you need to work with ...

... the ***task***

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_tasks)

<h4 id="BlurrUtil.get_tasks" class="doc_header"><code>BlurrUtil.get_tasks</code><a href="__main__.py#L60" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_tasks</code>(**`arch`**:`str`=*`None`*)

This method can be used to get a list of all tasks supported by your transformers install, or
just those available to a specific architecture

**Parameters:**


 - **`arch`** : *`<class 'str'>`*, *optional*	<p>A transformer architecture (e.g., 'bert')</p>



In [ ]:
print(BLURR.get_tasks())
print('')
print(BLURR.get_tasks('bart'))

['CTC', 'CausalLM', 'Classification', 'ConditionalGeneration', 'EntityClassification', 'EntityPairClassification', 'EntitySpanClassification', 'Generation', 'ImageClassification', 'LMHead', 'LMHeadModel', 'MaskedLM', 'MultipleChoice', 'NextSentencePrediction', 'PreTraining', 'QuestionAnswering', 'QuestionAnsweringSimple', 'RegionToPhraseAlignment', 'SequenceClassification', 'Teacher', 'TokenClassification', 'VisualReasoning', 'merLayer', 'merModel', 'merPreTrainedModel']

['CausalLM', 'ConditionalGeneration', 'QuestionAnswering', 'SequenceClassification']


... the ***architecture***

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_architectures)

<h4 id="BlurrUtil.get_architectures" class="doc_header"><code>BlurrUtil.get_architectures</code><a href="__main__.py#L72" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_architectures</code>()





In [ ]:
print(BLURR.get_architectures())

['albert', 'albert_fast', 'bart', 'bart_fast', 'barthez', 'barthez_fast', 'bert', 'bert_fast', 'bert_generation', 'bert_japanese', 'bertweet', 'big_bird', 'big_bird_fast', 'bigbird_pegasus', 'blenderbot', 'blenderbot_small', 'byt5', 'camembert', 'camembert_fast', 'canine', 'clip', 'clip_fast', 'convbert', 'convbert_fast', 'cpm', 'ctrl', 'deberta', 'deberta_fast', 'deberta_v2', 'deit', 'detr', 'distilbert', 'distilbert_fast', 'dpr', 'dpr_fast', 'electra', 'electra_fast', 'encoder_decoder', 'flaubert', 'fsmt', 'funnel', 'funnel_fast', 'gpt2', 'gpt2_fast', 'gpt_neo', 'herbert', 'herbert_fast', 'hubert', 'ibert', 'layoutlm', 'layoutlm_fast', 'led', 'led_fast', 'longformer', 'longformer_fast', 'luke', 'lxmert', 'lxmert_fast', 'm2m_100', 'marian', 'mbart', 'mbart50', 'mbart50_fast', 'mbart_fast', 'megatron_bert', 'mmbt', 'mobilebert', 'mobilebert_fast', 'mpnet', 'mpnet_fast', 'mt5', 'openai', 'openai_fast', 'pegasus', 'pegasus_fast', 'phobert', 'prophetnet', 'rag', 'reformer', 'reformer_fast

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_model_architecture)

<h4 id="BlurrUtil.get_model_architecture" class="doc_header"><code>BlurrUtil.get_model_architecture</code><a href="__main__.py#L91" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_model_architecture</code>(**`model_name_or_enum`**)

Get the architecture for a given model name / enum

**Parameters:**


 - **`model_name_or_enum`** : *`<class 'inspect._empty'>`*


In [ ]:
print(BLURR.get_model_architecture('RobertaForSequenceClassification'))

roberta


... and lastly the ***models*** (optionally for a given task and/or architecture)

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_models)

<h4 id="BlurrUtil.get_models" class="doc_header"><code>BlurrUtil.get_models</code><a href="__main__.py#L78" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_models</code>(**`arch`**:`str`=*`None`*, **`task`**:`str`=*`None`*)

The transformer models available for use (optional: by architecture | task)

**Parameters:**


 - **`arch`** : *`<class 'str'>`*, *optional*	<p>A transformer architecture (e.g., 'bert')</p>


 - **`task`** : *`<class 'str'>`*, *optional*	<p>A transformer task (e.g., 'TokenClassification')</p>



In [ ]:
print(L(BLURR.get_models())[:5])

['AdaptiveEmbedding', 'AlbertForMaskedLM', 'AlbertForMultipleChoice', 'AlbertForPreTraining', 'AlbertForQuestionAnswering']


In [ ]:
print(BLURR.get_models(arch='bert')[:5])

['BertForMaskedLM', 'BertForMultipleChoice', 'BertForNextSentencePrediction', 'BertForPreTraining', 'BertForQuestionAnswering']


In [ ]:
print(BLURR.get_models(task='TokenClassification')[:5])

['AlbertForTokenClassification', 'BertForTokenClassification', 'BigBirdForTokenClassification', 'CamembertForTokenClassification', 'CanineForTokenClassification']


In [ ]:
print(BLURR.get_models(arch='bert', task='TokenClassification'))

['BertForTokenClassification']


Here we define some helpful enums to make it easier to get at the *task and architecture* you're looking for.

In [ ]:
#export
HF_TASKS = Enum('HF_TASKS_ALL', BLURR.get_tasks())

In [ ]:
print('--- all tasks ---')
print(L(HF_TASKS))

--- all tasks ---
[<HF_TASKS_ALL.CTC: 1>, <HF_TASKS_ALL.CausalLM: 2>, <HF_TASKS_ALL.Classification: 3>, <HF_TASKS_ALL.ConditionalGeneration: 4>, <HF_TASKS_ALL.EntityClassification: 5>, <HF_TASKS_ALL.EntityPairClassification: 6>, <HF_TASKS_ALL.EntitySpanClassification: 7>, <HF_TASKS_ALL.Generation: 8>, <HF_TASKS_ALL.ImageClassification: 9>, <HF_TASKS_ALL.LMHead: 10>, <HF_TASKS_ALL.LMHeadModel: 11>, <HF_TASKS_ALL.MaskedLM: 12>, <HF_TASKS_ALL.MultipleChoice: 13>, <HF_TASKS_ALL.NextSentencePrediction: 14>, <HF_TASKS_ALL.PreTraining: 15>, <HF_TASKS_ALL.QuestionAnswering: 16>, <HF_TASKS_ALL.QuestionAnsweringSimple: 17>, <HF_TASKS_ALL.RegionToPhraseAlignment: 18>, <HF_TASKS_ALL.SequenceClassification: 19>, <HF_TASKS_ALL.Teacher: 20>, <HF_TASKS_ALL.TokenClassification: 21>, <HF_TASKS_ALL.VisualReasoning: 22>, <HF_TASKS_ALL.merLayer: 23>, <HF_TASKS_ALL.merModel: 24>, <HF_TASKS_ALL.merPreTrainedModel: 25>]


In [ ]:
HF_TASKS.Classification

<HF_TASKS_ALL.Classification: 3>

In [ ]:
#export
HF_ARCHITECTURES = Enum('HF_ARCHITECTURES', BLURR.get_architectures())

In [ ]:
print(L(HF_ARCHITECTURES)[:5])

[<HF_ARCHITECTURES.albert: 1>, <HF_ARCHITECTURES.albert_fast: 2>, <HF_ARCHITECTURES.bart: 3>, <HF_ARCHITECTURES.bart_fast: 4>, <HF_ARCHITECTURES.barthez: 5>]


### To get all your Hugging Face objects (arch, config, tokenizer, and model)

In [ ]:
show_doc(BlurrUtil(BlurrUtil).get_hf_objects)

<h4 id="BlurrUtil.get_hf_objects" class="doc_header"><code>BlurrUtil.get_hf_objects</code><a href="__main__.py#L99" class="source_link" style="float:right">[source]</a></h4>

> <code>BlurrUtil.get_hf_objects</code>(**`pretrained_model_name_or_path`**, **`model_cls`**, **`config`**=*`None`*, **`tokenizer_cls`**=*`None`*, **`config_kwargs`**=*`{}`*, **`tokenizer_kwargs`**=*`{}`*, **`model_kwargs`**=*`{}`*, **`cache_dir`**=*`None`*)

Given at minimum a `pretrained_model_name_or_path` and `model_cls (such as 
`AutoModelForSequenceClassification"), this method returns all the Hugging Face objects you need to train
a model using Blurr

**Parameters:**


 - **`pretrained_model_name_or_path`** : *`<class 'inspect._empty'>`*	<p>The name or path of the pretrained model</p>


 - **`model_cls`** : *`<class 'inspect._empty'>`*	<p>The model class you want to use (e.g., AutoModelFor<task>)</p>


 - **`config`** : *`<class 'NoneType'>`*, *optional*	<p>A specific configuration instance you want to use. If None, a configuration object will be instantiated
using the AutoConfig class along with any supplied `config_kwargs`</p>


 - **`tokenizer_cls`** : *`<class 'NoneType'>`*, *optional*	<p>A specific tokenizer instance you want to use.</p>


 - **`config_kwargs`** : *`<class 'dict'>`*, *optional*	<p>Any kwargs you want to pass to the `AutoConfig` (only used if you do NOT pass int a config above)</p>


 - **`tokenizer_kwargs`** : *`<class 'dict'>`*, *optional*	<p>Any kwargs you want to pass in the creation of your tokenizer</p>


 - **`model_kwargs`** : *`<class 'dict'>`*, *optional*	<p>Any kwargs you want to pass in the creation of your model</p>


 - **`cache_dir`** : *`<class 'NoneType'>`*, *optional*	<p>If you want to change the location Hugging Face objects are cached</p>



How to use:

In [ ]:
from transformers import AutoModelForMaskedLM
arch, config, tokenizer, model = BLURR.get_hf_objects("bert-base-cased-finetuned-mrpc",
                                                      model_cls=AutoModelForMaskedLM)

print(arch)
print(type(config))
print(type(tokenizer))
print(type(model))

bert
<class 'transformers.models.bert.configuration_bert.BertConfig'>
<class 'transformers.models.bert.tokenization_bert_fast.BertTokenizerFast'>
<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>


In [ ]:
from transformers import AutoModelForQuestionAnswering
arch, tokenizer, config, model = BLURR.get_hf_objects("fmikaelian/flaubert-base-uncased-squad",
                                                      model_cls=AutoModelForQuestionAnswering)

print(arch)
print(type(config))
print(type(tokenizer))
print(type(model))

In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction
arch, tokenizer, config, model = BLURR.get_hf_objects("bert-base-cased-finetuned-mrpc",
                                                      config=None,
                                                      tokenizer_cls=BertTokenizer, 
                                                      model_cls=BertForNextSentencePrediction)
print(arch)
print(type(config))
print(type(tokenizer)) 
print(type(model))

## Summary

Using the `BLURR` object is optional, as you're free to build your Hugging Face objects manually which sometimes may be required given your use case.  Most of the time, however, `BLURR` is sufficent to get you everything you need in one-line with a pretrained model name or path and task specific transformer model class.

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()